In [1]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

DEVICE = "cuda"


import torch, random, numpy as np
from transformers import set_seed

def set_all_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    set_seed(seed)
    torch.backends.cudnn.deterministic = True

set_all_seeds(9)

from safetensors import safe_open

In [4]:
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-256M-Instruct")
messages = [
    {
        "role": "user",
        "content": [
            # {"type": "image"},
            # {"type": "text", "text": "Can you describe the image?"}
            {"type": "text", "text": "What is life?"}
            # {"type": "text", "text": "A real-valued function f defined on the real line is called an even function if f(-t) = f(t) for each real number t. Prove that the set of even functions defined on the real line with the operations of addition and scalar multiplication defined in Example 3 is a vector space."}
        ]
    },
]

In [16]:
# Initialize model directly on CUDA without Flash Attention
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-256M-Instruct",
    torch_dtype=torch.float32,
    # _attn_implementation="flash_attention_2",  # Commented out Flash Attention
    device_map="cpu",
)
model.eval()


embeddings = {}
counter_token_pos = -1

def hook_fn(name, initial_layer: bool = False):
    def hook(module, input, output):
        global counter_token_pos
        if initial_layer:
            counter_token_pos += 1
        
        if isinstance(output, tuple):
            output = output[0]
        elif isinstance(output, torch.Tensor):
            pass
        else:
            print("Hook unknown type!!!", name, type(output))

        embeddings[name+f"_{counter_token_pos}"] = output.detach().cpu()

    return hook

# Register hooks for different layers
model.get_input_embeddings().register_forward_hook(hook_fn("input_embeddings", initial_layer=True))
for i in range(24):
    model.model.text_model.layers[i].input_layernorm.register_forward_hook(hook_fn(f"input_layernorm_d{i}"))
    model.model.text_model.layers[i].self_attn.register_forward_hook(hook_fn(f"self_attn_d{i}"))
    model.model.text_model.layers[i].self_attn.k_proj.register_forward_hook(hook_fn(f"self_attn_k_proj_d{i}"))
    model.model.text_model.layers[i].self_attn.v_proj.register_forward_hook(hook_fn(f"self_attn_v_proj_d{i}"))
    model.model.text_model.layers[i].self_attn.q_proj.register_forward_hook(hook_fn(f"self_attn_q_proj_d{i}"))
    model.model.text_model.layers[i].self_attn.o_proj.register_forward_hook(hook_fn(f"self_attn_o_proj_d{i}"))
    model.model.text_model.layers[i].post_attention_layernorm.register_forward_hook(hook_fn(f"post_layernorm_d{i}"))
    # model.model.text_model.layers[i].mlp.register_forward_hook(hook_fn(f"mlp_d{i}"))
    model.model.text_model.layers[i].mlp.gate_proj.register_forward_hook(hook_fn(f"mlp_gate_proj_d{i}"))
    model.model.text_model.layers[i].mlp.up_proj.register_forward_hook(hook_fn(f"mlp_up_proj_d{i}"))
    model.model.text_model.layers[i].mlp.down_proj.register_forward_hook(hook_fn(f"mlp_down_proj_d{i}"))
    model.model.text_model.layers[i].mlp.act_fn.register_forward_hook(hook_fn(f"mlp_act_fn_d{i}"))
    model.model.text_model.layers[i].register_forward_hook(hook_fn(f"layers_d{i}"))

type(model)

transformers.models.idefics3.modeling_idefics3.Idefics3ForConditionalGeneration

In [17]:
model

Idefics3ForConditionalGeneration(
  (model): Idefics3Model(
    (vision_model): Idefics3VisionTransformer(
      (embeddings): Idefics3VisionEmbeddings(
        (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
        (position_embedding): Embedding(1024, 768)
      )
      (encoder): Idefics3Encoder(
        (layers): ModuleList(
          (0-11): 12 x Idefics3EncoderLayer(
            (self_attn): Idefics3VisionAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
            (mlp): Idefics3VisionMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features=7

In [18]:
# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
# inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = processor(text=prompt, return_tensors="pt")
inputs = inputs.to("cpu")

print(inputs["input_ids"])
# Generate outputs
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,
        # repition_penalty=1.1,  # Apply repeat penalty
        output_scores=True,           # Return logits for each generated token
        return_dict_in_generate=True, # Return detailed output object
        do_sample=False,  # Use greedy decoding (highest logit)
    )

outputs.sequences[0]

tensor([[    1, 11126,    42,  1812,   314,  1029,    47, 49279,   198,  9519,
          9531,    42]])


tensor([    1, 11126,    42,  1812,   314,  1029,    47, 49279,   198,  9519,
         9531,    42,  5330,   314,   260,   768,  1070,  2775,   281,   451,
          905,    30,   657,   314,   260,   768,  1070,  2775,   281,   451,
          905,    30,   657,   314,   260,   768,  1070,  2775,   281,   451,
          905,    30,   657,   314,   260,   768,  1070,  2775,   281,   451,
          905,    30,   657,   314,   260,   768,  1070,  2775,   281,   451,
          905,    30,   657,   314,   260,   768,  1070,  2775,   281,   451,
          905,    30,   657,   314,   260,   768,  1070,  2775,   281,   451,
          905,    30,   657,   314,   260,   768,  1070,  2775,   281,   451,
          905,    30,   657,   314,   260,   768,  1070,  2775,   281,   451,
          905,    30,   657,   314,   260,   768,  1070,  2775,   281,   451,
          905,    30,   657,   314,   260,   768,  1070,  2775,   281,   451,
          905,    30,   657,   314,   260,   768,  1070,  2775, 

In [19]:
processor.tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

'User: What is life?\nAssistant: Life is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world. It is the most important thing in this world.

In [ ]:
import os
from safetensors.torch import save_file as save_safetensor

# Specify the layer index you want to save activations for
layer_idx = 0  # Change this to the desired layer index
token_pos = 10  # Change this to the desired token position if needed

# Keys for input and output activations in the embeddings dict
input_key = f"input_layernorm_d{layer_idx}_{token_pos}"
output_key = f"self_attn_q_proj_d{layer_idx}_{token_pos}"

# Retrieve tensors from embeddings dict
input_tensor = embeddings.get(input_key)
output_tensor = embeddings.get(output_key)

if input_tensor is None or output_tensor is None:
    raise ValueError(f"Could not find input or output tensor for layer {layer_idx} and token {token_pos}.")

# Prepare dict for safetensors
to_save = {
    "input": input_tensor,
    "output": output_tensor,
}

# Save to safetensors file
save_path = os.path.join("../validation_data", f"extracted_layer_activations.safetensors")
save_safetensor(to_save, save_path)
print(f"Saved activations to {save_path}")

Saved activations to ../../../tests/data/extracted_layer_activations.safetensors
